In [7]:
#IMPORT
try:
    import graphlab as gl
    import numpy as np
    import graphlab.aggregate as agg
    import matplotlib.pyplot as plt
    from matplotlib import rcParams
    import datetime as dt   
    from graphlab import degree_counting
    from graphlab import connected_components
    from graphlab import pagerank
    from graphlab import shortest_path
    from graphlab import triangle_counting
    from graphlab import label_propagation
    from graphlab import kcore
    from graphlab import graph_coloring
    print('success')
except:
    raise ImportError("Key libraries cannot be loaded.")

success


In [8]:
import os

transaction_data = '../code/graph-code/data/blocks_417500_424572/'

if os.path.exists('../code/graph-code/data/blocks_417500_424572'):
    sf = gl.SFrame(transaction_data)
    s = sf.sample(0.0001, seed=1)
    df = s.to_dataframe()
    df = df[df['input_address'].notnull()]
    df = df[df['output_address'].notnull()]
    df = df.fillna(0)
    sf_transactions = gl.SFrame(df)
    g = gl.SGraph().add_edges(sf_transactions, src_field='input_address', dst_field='output_address')
    print(len(sf))    
    print(len(df))
    print(g)
else:
    print('cant find data')

62418698
6260
SGraph({'num_edges': 6260, 'num_vertices': 10852})


In [9]:
#LOAD TAGGED DATA

import json
import pandas as pd

whoiswho = '../code/identity_scraper/output/whoiswho.json'
blockinfo = '../code/identity_scraper/output/b_info_out.json'
explorer = '../code/identity_scraper/output/explorer.json'

if os.path.exists('../code/identity_scraper/output/'):
    with open(whoiswho) as json_data:
        who = json.load(json_data)
    with open(blockinfo) as json_data:
        block = json.load(json_data) 
    with open(explorer) as json_data:
        explorer = json.load(json_data) 
else:
    print('cant find data')

total = who + block + explorer
print(len(total))

6021793


In [10]:
#CREATE TAG DATAFRAME
import re

categories = []
for i in total:
    addr = re.search("'(.*?)'", str(i)).group(0)
    num = re.search('(?<=\s)[0-9]', str(i)).group(0)
    addr.replace("'", "")
    t_dict = {
        'address': addr,
        'category': int(num)
    }
    categories.append(t_dict)

df_tags = pd.DataFrame.from_dict(categories)
df_tags['address'] = df_tags['address'].map(lambda x: str(x)[:-1])
df_tags['address'] = df_tags['address'].map(lambda x: str(x)[1:])
df_tags = df_tags.drop_duplicates()
print(df_tags.head(n=2))
print(len(df_tags))

                              address  category
0  15Mb3E3kcTqThNZitaYwWZ7PwZGQwXcUpU         1
1  13oPp2XG8PJJUbKUuhQ7ptMC3hnVcc1QzA         1
6015728


In [11]:
df = s.to_dataframe()
df = df[df['input_address'].notnull()]
df = df[df['output_address'].notnull()]

In [12]:
#ADD TAGS TO TRANSACTION RECORDS
df_transactions = df
df_transactions = (pd.merge(df_transactions, df_tags.groupby(['address', 'category'])['category'].size().unstack()
                        .add_prefix('input_')
                        .reset_index(), how='left', left_on = 'input_address', right_on = 'address')
                        .fillna(0))

df_transactions = (pd.merge(df_transactions, df_tags.groupby(['address', 'category'])['category'].size().unstack()
                        .add_prefix('output_')
                        .reset_index(), how='left', left_on = 'output_address', right_on = 'address')
                        .fillna(0))

In [18]:
print(df_transactions.columns)

Index([u'tx_hash', u'coinbase', u'block_hash', u'height', u'timestamp', u'id',
       u'value', u'script_type', u'output_address', u'input_address', u'year',
       u'month', u'day', u'close-price', u'address_x', u'input_0', u'input_1',
       u'input_2', u'input_3', u'input_4', u'input_5', u'input_6',
       u'address_y', u'output_0', u'output_1', u'output_2', u'output_3',
       u'output_4', u'output_5', u'output_6'],
      dtype='object')


In [20]:
print(df_transactions.head(5))

                                             tx_hash coinbase  \
0  74546e559319ec849668561c339d94d109059d531772ba...    False   
1  867426ba7ae467249395b308ed983296600dfaae1218f9...    False   
2  f42bc58e09f6599bbe8e00deb69746d4015f172ad9557c...    False   
3  3983c5d1fd10e23329d48e6b43043ca391efbf4bd156a6...    False   
4  4204f912856d9c67b605f9bf00bf35210d3eacc2b51ad0...    False   

                                          block_hash  height  \
0  00000000000000000066e02d77d96f495953f35899998c...  423934   
1  00000000000000000332a6f2704844bf0531a416ec50bb...  420414   
2  0000000000000000018ac2a18dcf812eaba901285aa8f4...  421973   
3  000000000000000003d7a64cfecc8fe3be68526dd24b99...  421683   
4  00000000000000000052236533a6da5bca91b0e7f564fd...  422513   

            timestamp   id      value script_type  \
0 2016-08-06 07:56:10    1   0.006712  pubkeyhash   
1 2016-07-12 11:14:29  185   0.000201  pubkeyhash   
2 2016-07-23 15:55:45    0   0.744930  pubkeyhash   
3 2016-07-21